In [1]:
import multiprocessing
import jieba
import nltk
import os
import time
from math import log
import json
import csv
import pandas as pd
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:939: UserWarning: could not find rc file; returning defaults
  warnings.warn(message)


In [2]:
tmp_catalog = '/Users/cecexu/Documents/cecexu/advertisement/cnews/'

In [3]:
feeds_file = tmp_catalog + 'douyin_ad.csv'

In [4]:
all_ads = tmp_catalog + 'all_ads.csv'

In [5]:
stopwords_file = tmp_catalog + 'stopwords1893.txt'

In [6]:
#read stop words
def read_stopwords(filepath):
    stopwords = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            stopwords.append(line.strip())
    return stopwords

In [7]:
stopwords_list = read_stopwords(stopwords_file)

In [8]:
#seperate sentence
def seg_sentence(sentence, stopwords):
    sentence_seged = jieba.cut(sentence.strip())
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += ' '
    return outstr

In [9]:
#read class and description from original dataset, and delete duplicate rows
from collections import defaultdict
account = []
def read_file(filename):
    outfile = []
    with open(filename, 'r', encoding='utf-8') as f:
        f_reader = csv.reader(f,delimiter = ',')
        next(f_reader, None)
        for line in f_reader:
            if line[1] not in account:
                account.append(line[1])
                outfile.append([line[0],line[1], line[2]])
            else:
                outfile.append([line[0],line[1], line[2]])
        outfile = pd.DataFrame(outfile)
    return outfile

In [10]:
out_feeds_df = read_file(feeds_file)

In [11]:
out_feeds_df.columns = ['class','account','desp']

In [12]:
out_feeds_df.head()

,class,account,desp
0,护肤美容,{痘博士},“一位痘友，一名顾问，一套方案”痘博士专业祛痘
1,护肤美容,{痘博士},如果你长痘还不会处理，那就预约痘博士祛痘套餐21.5
2,护肤美容,{痘博士},痘博士468祛痘套餐仅需21.5，每日仅限30名额
3,护肤美容,{痘博士},痘博士线上预约，线下体验，21.5体验面部祛痘套餐
4,护肤美容,{痘博士},分型祛痘，细致专业!21.5体验痘博士原价468元祛痘套餐


In [13]:
# split sentences and read label and corpus from classification data
def corpus_file(df):
    out_label = []
    out_corpus = []
    for i in range(len(df)):
        cut_words_act = seg_sentence(df['account'][i], stopwords_list)
        cut_words_desp = seg_sentence(df['desp'][i], stopwords_list)
        tmp = cut_words_act + cut_words_desp
        out_corpus.append([df['class'][i],df['account'][i], tmp])
    out_corpus = pd.DataFrame(out_corpus)
    return out_corpus

In [14]:
corpus_df = corpus_file(out_feeds_df)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/6z/15w3r5bs0g5cc8frnnxvpv9m0000gp/T/jieba.cache
Loading model cost 0.942 seconds.
Prefix dict has been built succesfully.


In [15]:
corpus_df.columns = ['class','account','desp']
corpus_df = corpus_df.drop_duplicates(subset=['class', 'desp'], keep=False)

In [16]:
corpus_df.groupby('class').count().reset_index()

,class,account,desp
0,家居家装,1578,1578
1,房地产,590,590
2,护肤美容,1009,1009
3,摄影,454,454
4,教育培训,1764,1764
5,文化娱乐,630,630
6,新闻,670,670
7,旅游出行,278,278
8,日用百货,189,189
9,服饰鞋包,367,367


In [17]:
#balance data 
corpus_df_1 = corpus_df[corpus_df['class']=='日用百货']
corpus_df_2 = corpus_df[corpus_df['class']=='旅游出行']

In [18]:
corpus_df = corpus_df.append(corpus_df_1)
corpus_df = corpus_df.append(corpus_df_1)
corpus_df = corpus_df.append(corpus_df_2)

In [19]:
len(corpus_df)

17671

In [20]:
corpus_df.groupby('class').count().reset_index()

,class,account,desp
0,家居家装,1578,1578
1,房地产,590,590
2,护肤美容,1009,1009
3,摄影,454,454
4,教育培训,1764,1764
5,文化娱乐,630,630
6,新闻,670,670
7,旅游出行,556,556
8,日用百货,567,567
9,服饰鞋包,367,367


In [21]:
#calculate tf-idf of the whole dataset
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1e-5) #drop df <1e-5, low frequency words
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus_df['desp']))
words = vectorizer.get_feature_names()
print(tfidf.toarray())
print(tfidf.shape[0], tfidf.shape[1])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
17671 16331


In [22]:
# #encode labels
# from sklearn import preprocessing
# clas_corpus = list(corpus_df['desp'])
# clas_label = list(corpus_df['class'])
# clas_account = list(corpus_df['account'])
# encoder = preprocessing.LabelEncoder()
# encoder_label = encoder.fit_transform(clas_label) 

In [23]:
tmp_feeds = corpus_df

In [24]:
tmp_feeds[0:5]

,class,account,desp
0,护肤美容,{痘博士},痘 博士 一位 痘友 一名 顾问 一套 方案 痘 博士 专业 祛痘
1,护肤美容,{痘博士},痘 博士 长 痘 预约 痘 博士 祛痘 套餐 21.5
2,护肤美容,{痘博士},痘 博士 痘 博士 468 祛痘 套餐 需 21.5 每日 仅限 30 名额
3,护肤美容,{痘博士},痘 博士 痘 博士 线上 预约 线下 体验 21.5 体验 面部 祛痘 套餐
4,护肤美容,{痘博士},痘 博士 分型 祛痘 细致 专业 21.5 体验 痘 博士 原价 468 元 祛痘 套餐


In [25]:
#split dataset to training and test data
def split_train_test(df, test_ratio):
    np.random.seed(100)
    shuffle_index = np.random.permutation(len(df))
    test_set_size=int(len(df)*test_ratio)
    test_indices =shuffle_index[:test_set_size]
    train_indices=shuffle_index[test_set_size:]
    return df.iloc[train_indices], df.iloc[test_indices]

In [28]:
clas_corpus

['痘 博士 一位 痘友 一名 顾问 一套 方案 痘 博士 专业 祛痘 ',
 '痘 博士 长 痘 预约 痘 博士 祛痘 套餐 21.5 ',
 '痘 博士 痘 博士 468 祛痘 套餐 需 21.5 每日 仅限 30 名额 ',
 '痘 博士 痘 博士 线上 预约 线下 体验 21.5 体验 面部 祛痘 套餐 ',
 '痘 博士 分型 祛痘 细致 专业 21.5 体验 痘 博士 原价 468 元 祛痘 套餐 ',
 '痘 博士 脸上 满是 痘痘 难看 痘 博士 21.5 面部 祛痘 ',
 '痘 博士 告诉 秘密 领券后 买 衣服 不到 50 太值 ',
 '痘 博士 领券后 下单 每套 衣服 不到 50 元 ',
 '痘 博士 同事 新 衣服 买 领完券 一套 不到 50 元 ',
 '痘 博士 告诉 秘密 领券后 买 衣服 50 元 太值 ',
 '痘 博士 同事 衣服 买 一套 50 元 超 划算 ',
 '痘 博士 秋季 福利 买 一套 衣服 50 元 包邮 ',
 '痘 博士 同事 新 衣服 买 领完券 一套 50 元 ',
 '痘 博士 脸上 痘痘 毛孔 清洁 预约 21.5 面部 祛痘 套餐 ',
 '痘 博士 脸上 痘痘 反复 长 不停 快来 痘 博士 预约 21.5 祛痘 套餐 ',
 '痘 博士 祛痘 脸上 痘痘 迟迟 预约 痘 博士 21.5 面部 祛痘 ',
 '痘 博士 用手挤 痘痘 痘 博士 祛痘 套餐 需 21.5 ',
 '中国 合伙人 阿修罗 阿修罗 ',
 '中国 合伙人 中国 合伙人 12 月 18 日致 时代 创业者 ',
 '痘 博士 限时 预约 21.5 元 体验 项目 标准化 祛痘 服务 流程 全程 约 45 分钟 ',
 '痘 博士 限时 预约 想要 正确 祛痘 听 痘 博士 体验 专业 祛痘 谱 ',
 '痘 博士 限时 预约 祛痘 成功 快来 痘 博士 体验 专业 祛痘 需 11 元 ',
 '痘 博士 限时 预约 长痘 体验 痘 博士 专业 祛痘 套餐 ',
 '痘 博士 限时 预约 说 男生 不用 祛痘 痘 博士 专业 祛痘 体验 男女 限 ',
 '痘 博士 限时 预约 11 月 体验 痘 博士 专业 祛痘 需 11 元 ',
 '痘 博士 限时 预约 说 男生 不用 祛痘 11 元 专业 祛痘 套餐 男女

In [29]:
#tmp_feeds = clas_corpus[clas_corpus['class'].isin(clas)].reset_index(drop = True)

In [30]:
len(tmp_feeds)

17671

In [31]:
tmp = tmp_feeds.groupby(['class']).count().reset_index()

In [32]:
tmp

,class,account,desp
0,家居家装,1578,1578
1,房地产,590,590
2,护肤美容,1009,1009
3,摄影,454,454
4,教育培训,1764,1764
5,文化娱乐,630,630
6,新闻,670,670
7,旅游出行,556,556
8,日用百货,567,567
9,服饰鞋包,367,367


In [33]:
tmp_clas1 = list(tmp.loc[tmp['desp'] > 900]['class'])
tmp_clas1

['家居家装', '护肤美容', '教育培训', '游戏', '珠宝腕表', '社交', '综合电商', '运动健身']

In [34]:
#x = [train_corpus[i].split() for i in range(len(train_corpus))]

In [35]:
#check median and average length of each sentences of corpus
from statistics import mean, median
#max(len(all_corpus[i].split()) for i in range(len(all_corpus)))
a =[]
for i in range(len(all_corpus)):
    a.append(len(all_corpus[i].split()))
print(mean(a), median(a), max(a))

10.873691358723333 11 39


In [59]:
all_corpus = list(tmp_feeds['desp'])
all_label = list(tmp_feeds['class'])

In [36]:
#encode labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder_label = encoder.fit_transform(all_label) 

In [60]:
#split data to train, and test datasets
np.random.seed(100)
test_ratio = 0.1
shuffle_index = np.random.permutation(len(all_label))
test_set_size=int(len(all_label)*test_ratio)
test_indices =shuffle_index[:test_set_size]
train_indices =shuffle_index[test_set_size:]

In [61]:
train_corpus = tmp_feeds.iloc[train_indices]['desp']
test_corpus = tmp_feeds.iloc[test_indices]['desp']
train_label = encoder_label[train_indices]
test_label = encoder_label[test_indices]

In [62]:
from gensim.models.word2vec import Word2Vec
from gensim import corpora,models,similarities
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)

In [63]:
#creat corpus format for word2vec
new_corpus = []
for i in all_corpus:
    new_corpus.append(i.split())

In [64]:
model = Word2Vec(new_corpus, size=128, window=5, min_count=1, workers=4, sg = 1)
model.save('word2vec.model')
model = Word2Vec.load('word2vec.model')

2019-03-05 20:51:09,246:INFO:collecting all words and their counts
2019-03-05 20:51:09,247:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-05 20:51:09,273:INFO:PROGRESS: at sentence #10000, processed 110750 words, keeping 13420 word types
2019-03-05 20:51:09,291:INFO:collected 17860 word types from a corpus of 192149 raw words and 17671 sentences
2019-03-05 20:51:09,292:INFO:Loading a fresh vocabulary
2019-03-05 20:51:09,330:INFO:effective_min_count=1 retains 17860 unique words (100% of original 17860, drops 0)
2019-03-05 20:51:09,331:INFO:effective_min_count=1 leaves 192149 word corpus (100% of original 192149, drops 0)
2019-03-05 20:51:09,379:INFO:deleting the raw counts dictionary of 17860 items
2019-03-05 20:51:09,380:INFO:sample=0.001 downsamples 45 most-common words
2019-03-05 20:51:09,381:INFO:downsampling leaves estimated 180871 word corpus (94.1% of prior 192149)
2019-03-05 20:51:09,427:INFO:estimated required memory for 17860 words and 128 dimen

In [65]:
model.wv.save_word2vec_format("./word2Vec" + ".txt", binary=True) 

2019-03-05 20:51:12,452:INFO:storing 17860x128 projection weights into ./word2Vec.txt


In [66]:
print(model['南京'])

[-0.8024654   0.17015441  0.06040993  0.2920847   0.28968036  0.40784955
 -0.10929088  0.45116135  0.12550579 -0.47209468  0.07671929 -0.12314268
 -0.01855885  0.5136108   0.57675296  0.2992371  -0.22868152  0.76708835
  0.23987773 -0.45560846  0.18431617  0.03545198 -0.2994968  -0.10449723
  0.33923417 -0.3591977   0.03426103  0.2579027  -0.5883679   0.47153306
 -0.3159799  -0.19116783 -0.00696525 -0.12106136  0.28336772 -0.09274793
 -0.2988049  -0.10332027 -0.06201471 -0.33114064  0.16692132  0.1880747
  0.6184898   0.18304761 -0.09166851  0.48769     0.27704665  0.5011212
 -0.03390543 -0.12400928  0.29433352 -0.43131632 -0.08109356  0.20963629
  0.1520722  -0.13324624  0.7633223   0.34309152 -0.63521856  0.65662384
 -0.06383843  0.18997437  0.08730643  0.34883577  0.12755282  0.16018152
  0.25193197  0.01846722 -0.3543096   0.8570626   0.00798788  0.31023884
 -0.5928756  -0.5513693   0.44266504  0.28218612 -0.35626754  0.03609127
 -0.27903372 -0.34457654 -0.3589382  -0.16578852  0.3

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [67]:
model.wv.most_similar('南京', topn=3)

2019-03-05 20:51:14,038:INFO:precomputing L2-norms of word weight vectors
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('米兰', 0.9688922762870789),
 ('婚纱', 0.9665706753730774),
 ('尊荣', 0.957295298576355)]

In [68]:
model.wv.similarity('南京','装修')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.51736253

In [70]:
#model1: classification by basic cnn model and keras
#define the common length of a sentence as 11, and convert sentences to matrix padding_size*vec_size
def transform_to_matrix(corpus, padding_size, vec_size):
    res = []
    for sen in corpus:
        sen = sen.split()
        matrix = []
        for i in range(padding_size):
            try:
                matrix.append(model[sen[i]])
            except:
                matrix.append([0] * vec_size)  #补零
        res.append(matrix)
    return res 

In [71]:
res_corpus = transform_to_matrix(all_corpus, padding_size = 40, vec_size = 128)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [72]:
#select train dataset and test dataset
X_train = transform_to_matrix(train_corpus, 40, 128)
X_test = transform_to_matrix(test_corpus, 40, 128)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [73]:
#convert to format of np for processing easily
X_train = np.array(X_train)
X_test = np.array(X_test)
print(X_train.shape, X_test.shape)

(15904, 40, 128) (1767, 40, 128)


In [82]:
#cnn by keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Convolution1D, MaxPooling1D
from keras.layers.core import Dense, Dropout, Activation, Flatten

#set parameters:
n_filter = 16
batch_size = 32
filter_length = 3
wid_filter = 40
len_filter = 3
nb_epoch = 10
n_pool =1

#build up a new sequential model
model = Sequential()
#By default, Convolution2D (https://keras.io/layers/convolutional/) expects the input to be 
#in the format (samples, rows, cols, channels), which is "channels-last". Your data seems to 
#be in the format (samples, channels, rows, cols). You should be able to fix this using the 
#optional keyword data_format = 'channels_first' when declaring the Convolution2D layer.
model.add(Convolution1D(n_filter, 4, input_shape = (40,128),\
                        data_format = 'channels_first'))
model.add(Activation('relu'))
model.add(Convolution1D(n_filter, 4))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size = 1))
model.add(Dropout(0.5))
model.add(Flatten())

In [83]:
#connected to ANN
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Dense(20)) #define output dimensions
model.add(Activation('softmax'))

In [84]:
from keras.utils import to_categorical
y_train = to_categorical(train_label,20)
y_test = to_categorical(test_label,20)

In [77]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [78]:
model.fit(X_train, y_train, batch_size=batch_size, epochs= 10, verbose=0)

In [79]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

In [80]:
loss

0.12927872992010528

In [81]:
accuracy

0.9626485568760611

In [51]:
#model 2: Building up model of sentences classification by paper of kim and tflearn.
#building convolutional network
import tensorflow as tf
from tflearn.data_utils import to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
import tflearn
import pickle

In [66]:
X_train.shape

(15904, 40, 128)

In [67]:
X_test.shape

(1767, 40, 128)

In [52]:
y_train = to_categorical(train_label, nb_classes = 20)
y_test = to_categorical(test_label, nb_classes = 20)

In [53]:
#use pre-trained word2vec +cnn model
#cnn by tflearn, building convolutional network

##[None, 100] `input_data` is used as a data entry (placeholder) of a network. 
#This placeholder will be feeded with data when training
network = input_data(shape = [None, 40, 128], name = 'input')

#[None, 30,128].embedding layer for a sequence of ids. network: Incoming 2-D Tensor. 
#input_dim: vocabulary size, oput_dim:vector size
#network = tflearn.embedding(network, input_dim = 19112, output_dim = 128)

In [58]:
#use td-idf + cnn model
network = input_data(shape = [None, 40], name = 'input')
network = tflearn.embedding(network, input_dim = 17671, output_dim = 128)

In [59]:
# ## [batch_size, vec_size, nb_filters]. padding:"VALID",only ever drops the right-most columns
branch1 = conv_1d(network, 128, 3, padding = 'valid', activation = 'relu', regularizer = 'L2')
branch2 = conv_1d(network, 128, 4, padding = 'valid', activation = 'relu', regularizer = 'L2')
branch3 = conv_1d(network, 128, 5, padding = 'valid', activation = 'relu', regularizer = 'L2')
network = merge([branch1, branch2, branch3], mode = 'concat', axis = 1)
network = tf.expand_dims(network, 2)
network = global_max_pool(network)
network = dropout(network, 0.5) #[batch_size, pooled dim]
network = fully_connected(network, 20, activation = 'softmax')
network = regression(network, optimizer = 'adam', learning_rate = 0.001, loss = 'categorical_crossentropy', name = 'target')

In [60]:
#training
model = tflearn.DNN(network, tensorboard_verbose=0)

In [61]:
model.fit(X_train, y_train, n_epoch = 10, shuffle=True, validation_set=(X_test, y_test), \
          show_metric=True, batch_size=256)

IndexError: list index out of range

In [120]:
# for i in range(len(weight)):
#     print(u"-------这里输出第",i,u"类文本的词语tf-idf权重------")
#     for j in range(len(word)):
#         print(word[j],weight[i][j])

In [262]:
np.savetxt('all_corpus.txt', all_corpus, delimiter=',', fmt='%s', header = 'desp', comments='')